In [1]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import time
import re

In [2]:
df = pd.read_csv(r'steamgames_test.csv')
df 

,Title,Original Price,Discounted Price,Release Date,Link
0,Baldur's Gate 3,$29.99,$29.99,"3 Aug, 2023",https://store.steampowered.com/app/1086940/Bal...
1,Counter-Strike: Global Offensive,$14.99,$14.99,"21 Aug, 2012",https://store.steampowered.com/app/730/Counter...
2,Apex Legends™,Free,Free,"4 Nov, 2020",https://store.steampowered.com/app/1172470/Ape...
3,Forza Horizon 5,$34.78,$17.39,"8 Nov, 2021",https://store.steampowered.com/app/1551360/For...
4,Call of Duty®,Free,Free,"27 Oct, 2022",https://store.steampowered.com/app/1938090/Cal...
...,...,...,...,...,...
81293,The Front,Free,Free,To be announced,https://store.steampowered.com/app/2285150/The...
81294,The Bornless,Free,Free,To be announced,https://store.steampowered.com/app/2411280/The...
81295,Winter Memories,Free,Free,Coming soon,https://store.steampowered.com/app/2495450/Win...
81296,Call of Duty®: Modern Warfare® III,$69.99,$69.99,Coming soon,https://store.steampowered.com/app/2519060/Cal...


In [3]:
extracted_info = []
processed_indices = []

for index, row in df.iterrows():
    if index in processed_indices:
        continue  

    game_url = row['Link']

    try:
        response = requests.get(game_url)
        response.raise_for_status()  
        html_content = response.text
        soup = BeautifulSoup(html_content, 'html.parser')
              
        game_description = soup.find('div', class_='game_description_snippet')
        game_description = game_description.text.strip() if game_description else None
        
        reviews_elems = soup.find_all('div', class_='user_reviews_summary_row')
        recent_reviews_summary = None
        recent_reviews_number = None
        all_reviews_summary = None
        all_reviews_number = None
        
        if len(reviews_elems) >= 1:
            recent_summary_elem = reviews_elems[0].find('span', class_='game_review_summary')
            recent_number_elem = reviews_elems[0].find('span', class_='nonresponsive_hidden responsive_reviewdesc')
            if recent_summary_elem and recent_number_elem:
                recent_reviews_summary = recent_summary_elem.text
                recent_reviews_number = recent_number_elem.text.strip()
        
        if len(reviews_elems) >= 2:
            all_summary_elem = reviews_elems[1].find('span', class_='game_review_summary')
            all_number_elem = reviews_elems[1].find('span', class_='nonresponsive_hidden responsive_reviewdesc')
            if all_summary_elem and all_number_elem:
                all_reviews_summary = all_summary_elem.text
                all_reviews_number = all_number_elem.text.strip()
        
        developer_elem = soup.find('div', class_='dev_row')
        developer = developer_elem.find('div', class_='summary').text.strip() if developer_elem else None
        
        publisher_label_elem = soup.find('b', string='Publisher:')
        publisher = None
        if publisher_label_elem:
            publisher = publisher_label_elem.find_next_sibling('a')
            publisher = publisher.text.strip() if publisher else None
        
        language_rows = soup.select('table.game_language_options tr')[1:] 
        supported_languages = [row.find('td', class_='ellipsis').text.strip() for row in language_rows]

        popular_tags_elems = soup.find_all('a', class_='app_tag')
        popular_tags = [tag.get_text(strip=True) for tag in popular_tags_elems]

        features_elems = soup.find_all('a', class_='game_area_details_specs_ctn')
        game_features = [feature.find('div', class_='label').get_text(strip=True) for feature in features_elems]

        minimum_requirements_elem = soup.find('strong', string='Minimum:')
        if minimum_requirements_elem:
            minimum_requirements = minimum_requirements_elem.find_next('ul', class_='bb_ul')
            minimum_requirements = minimum_requirements.get_text('\n').strip() if minimum_requirements else None
            minimum_requirements = minimum_requirements.replace('\n', ' | ')
        else:
            minimum_requirements = None

        extracted_info.append({
            'Game': row['Title'],
            'Game Description': game_description,
            'Recent Reviews Summary': recent_reviews_summary,
            'All Reviews Summary': all_reviews_summary,
            'Recent Reviews Number': recent_reviews_number,
            'All Reviews Number': all_reviews_number,
            'Developer': developer,
            'Publisher': publisher,
            'Supported Languages': supported_languages,
            'Popular Tags': popular_tags,
            'Game Features': game_features,
            'Minimum Requirements': minimum_requirements
        })
        
        processed_indices.append(index)  

        time.sleep(0.2)  

        if len(processed_indices) % 50 == 0:
            partial_df = pd.DataFrame(extracted_info)
            partial_df.to_csv('game_details_partial.csv', index=False)
            print(f"Partial game details saved to game_details_partial.csv (Iteration {index + 1}/{len(df)})")

    except Exception as e:
        print("Error processing game:", row['Title'])
        print("Error details:", str(e))

full_df = pd.DataFrame(extracted_info)

full_df.to_csv('game_details_full.csv', index=False)
print("Game details saved to game_details_full.csv")

Partial game details saved to game_details_partial.csv (Iteration 50/81298)
Partial game details saved to game_details_partial.csv (Iteration 100/81298)
Partial game details saved to game_details_partial.csv (Iteration 150/81298)
Partial game details saved to game_details_partial.csv (Iteration 200/81298)
Error processing game: Stray Gods: The Roleplaying Musical
Error details: HTTPSConnectionPool(host='store.steampowered.com', port=443): Max retries exceeded with url: /app/1920780/Stray_Gods_The_Roleplaying_Musical/?snr=1_7_7_230_150_5 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000022010BCC6D0>, 'Connection to store.steampowered.com timed out. (connect timeout=None)'))
Partial game details saved to game_details_partial.csv (Iteration 251/81298)
Partial game details saved to game_details_partial.csv (Iteration 301/81298)
Partial game details saved to game_details_partial.csv (Iteration 351/81298)
Partial game details saved to game_details_partial.cs

Partial game details saved to game_details_partial.csv (Iteration 4116/81298)
Partial game details saved to game_details_partial.csv (Iteration 4166/81298)
Partial game details saved to game_details_partial.csv (Iteration 4216/81298)
Partial game details saved to game_details_partial.csv (Iteration 4266/81298)
Partial game details saved to game_details_partial.csv (Iteration 4316/81298)
Partial game details saved to game_details_partial.csv (Iteration 4366/81298)
Partial game details saved to game_details_partial.csv (Iteration 4416/81298)
Partial game details saved to game_details_partial.csv (Iteration 4466/81298)
Partial game details saved to game_details_partial.csv (Iteration 4516/81298)
Partial game details saved to game_details_partial.csv (Iteration 4566/81298)
Partial game details saved to game_details_partial.csv (Iteration 4616/81298)
Partial game details saved to game_details_partial.csv (Iteration 4666/81298)
Error processing game: Driver® Parallel Lines
Error details: 'No

Partial game details saved to game_details_partial.csv (Iteration 8479/81298)
Partial game details saved to game_details_partial.csv (Iteration 8529/81298)
Partial game details saved to game_details_partial.csv (Iteration 8579/81298)
Partial game details saved to game_details_partial.csv (Iteration 8629/81298)
Partial game details saved to game_details_partial.csv (Iteration 8679/81298)
Partial game details saved to game_details_partial.csv (Iteration 8729/81298)
Partial game details saved to game_details_partial.csv (Iteration 8779/81298)
Partial game details saved to game_details_partial.csv (Iteration 8829/81298)
Partial game details saved to game_details_partial.csv (Iteration 8879/81298)
Partial game details saved to game_details_partial.csv (Iteration 8929/81298)
Partial game details saved to game_details_partial.csv (Iteration 8979/81298)
Partial game details saved to game_details_partial.csv (Iteration 9029/81298)
Error processing game: Precipice of Darkness, Episode One
Error 

Partial game details saved to game_details_partial.csv (Iteration 13037/81298)
Error processing game: Waking Mars
Error details: 'NoneType' object has no attribute 'replace'
Partial game details saved to game_details_partial.csv (Iteration 13088/81298)
Partial game details saved to game_details_partial.csv (Iteration 13138/81298)
Partial game details saved to game_details_partial.csv (Iteration 13188/81298)
Partial game details saved to game_details_partial.csv (Iteration 13238/81298)
Partial game details saved to game_details_partial.csv (Iteration 13288/81298)
Partial game details saved to game_details_partial.csv (Iteration 13338/81298)
Partial game details saved to game_details_partial.csv (Iteration 13388/81298)
Partial game details saved to game_details_partial.csv (Iteration 13438/81298)
Error processing game: Superbrothers: Sword & Sworcery EP
Error details: 'NoneType' object has no attribute 'replace'
Partial game details saved to game_details_partial.csv (Iteration 13489/8129

Partial game details saved to game_details_partial.csv (Iteration 17794/81298)
Partial game details saved to game_details_partial.csv (Iteration 17844/81298)
Partial game details saved to game_details_partial.csv (Iteration 17894/81298)
Partial game details saved to game_details_partial.csv (Iteration 17944/81298)
Error processing game: Pineapple Smash Crew 
Error details: 'NoneType' object has no attribute 'replace'
Partial game details saved to game_details_partial.csv (Iteration 17995/81298)
Partial game details saved to game_details_partial.csv (Iteration 18045/81298)
Partial game details saved to game_details_partial.csv (Iteration 18095/81298)
Partial game details saved to game_details_partial.csv (Iteration 18145/81298)
Partial game details saved to game_details_partial.csv (Iteration 18195/81298)
Error processing game: Rag Doll Kung Fu
Error details: 'NoneType' object has no attribute 'replace'
Partial game details saved to game_details_partial.csv (Iteration 18246/81298)
Parti

...

Partial game details saved to game_details_partial.csv (Iteration 60651/81298)
Partial game details saved to game_details_partial.csv (Iteration 60701/81298)
Partial game details saved to game_details_partial.csv (Iteration 60751/81298)
Partial game details saved to game_details_partial.csv (Iteration 60801/81298)
Partial game details saved to game_details_partial.csv (Iteration 60851/81298)
Partial game details saved to game_details_partial.csv (Iteration 60901/81298)
Partial game details saved to game_details_partial.csv (Iteration 60951/81298)
Partial game details saved to game_details_partial.csv (Iteration 61001/81298)
Partial game details saved to game_details_partial.csv (Iteration 61051/81298)
Partial game details saved to game_details_partial.csv (Iteration 61101/81298)
Partial game details saved to game_details_partial.csv (Iteration 61151/81298)
Partial game details saved to game_details_partial.csv (Iteration 61201/81298)
Partial game details saved to game_details_partial.c

Partial game details saved to game_details_partial.csv (Iteration 65851/81298)
Partial game details saved to game_details_partial.csv (Iteration 65901/81298)
Partial game details saved to game_details_partial.csv (Iteration 65951/81298)
Partial game details saved to game_details_partial.csv (Iteration 66001/81298)
Partial game details saved to game_details_partial.csv (Iteration 66051/81298)
Partial game details saved to game_details_partial.csv (Iteration 66101/81298)
Error processing game: Beijing 2008™ - The Official Video Game of the Olympic Games
Error details: 'NoneType' object has no attribute 'replace'
Partial game details saved to game_details_partial.csv (Iteration 66152/81298)
Partial game details saved to game_details_partial.csv (Iteration 66202/81298)
Partial game details saved to game_details_partial.csv (Iteration 66252/81298)
Partial game details saved to game_details_partial.csv (Iteration 66302/81298)
Partial game details saved to game_details_partial.csv (Iteration 

Partial game details saved to game_details_partial.csv (Iteration 70605/81298)
Partial game details saved to game_details_partial.csv (Iteration 70655/81298)
Partial game details saved to game_details_partial.csv (Iteration 70705/81298)
Partial game details saved to game_details_partial.csv (Iteration 70755/81298)
Partial game details saved to game_details_partial.csv (Iteration 70805/81298)
Partial game details saved to game_details_partial.csv (Iteration 70855/81298)
Partial game details saved to game_details_partial.csv (Iteration 70905/81298)
Partial game details saved to game_details_partial.csv (Iteration 70955/81298)
Partial game details saved to game_details_partial.csv (Iteration 71005/81298)
Partial game details saved to game_details_partial.csv (Iteration 71055/81298)
Partial game details saved to game_details_partial.csv (Iteration 71105/81298)
Partial game details saved to game_details_partial.csv (Iteration 71155/81298)
Partial game details saved to game_details_partial.c

Partial game details saved to game_details_partial.csv (Iteration 75706/81298)
Partial game details saved to game_details_partial.csv (Iteration 75756/81298)
Partial game details saved to game_details_partial.csv (Iteration 75806/81298)
Partial game details saved to game_details_partial.csv (Iteration 75856/81298)
Partial game details saved to game_details_partial.csv (Iteration 75906/81298)
Partial game details saved to game_details_partial.csv (Iteration 75956/81298)
Partial game details saved to game_details_partial.csv (Iteration 76006/81298)
Partial game details saved to game_details_partial.csv (Iteration 76056/81298)
Partial game details saved to game_details_partial.csv (Iteration 76106/81298)
Partial game details saved to game_details_partial.csv (Iteration 76156/81298)
Partial game details saved to game_details_partial.csv (Iteration 76206/81298)
Partial game details saved to game_details_partial.csv (Iteration 76256/81298)
Partial game details saved to game_details_partial.c

Partial game details saved to game_details_partial.csv (Iteration 80906/81298)
Partial game details saved to game_details_partial.csv (Iteration 80956/81298)
Partial game details saved to game_details_partial.csv (Iteration 81006/81298)
Partial game details saved to game_details_partial.csv (Iteration 81056/81298)
Partial game details saved to game_details_partial.csv (Iteration 81106/81298)
Partial game details saved to game_details_partial.csv (Iteration 81156/81298)
Partial game details saved to game_details_partial.csv (Iteration 81206/81298)
Partial game details saved to game_details_partial.csv (Iteration 81256/81298)
Game details saved to game_details_full.csv


In [4]:
full_df = pd.read_csv("game_details_full.csv")
full_df

,Game,Game Description,Recent Reviews Summary,All Reviews Summary,Recent Reviews Number,All Reviews Number,Developer,Publisher,Supported Languages,Popular Tags,Game Features,Minimum Requirements
0,Baldur's Gate 3,"Baldur’s Gate 3 is a story-rich, party-based R...",Overwhelmingly Positive,Very Positive,"- 96% of the 128,900 user reviews in the last ...","- 94% of the 188,617 user reviews for this gam...",Larian Studios,Larian Studios,"['English', 'French', 'German', 'Spanish - Spa...","['RPG', 'Choices Matter', 'Character Customiza...","['Single-player', 'Online Co-op', 'LAN Co-op',...",Requires a 64-bit processor and operating syst...
1,Counter-Strike: Global Offensive,Counter-Strike: Global Offensive (CS: GO) expa...,Very Positive,Very Positive,"- 89% of the 75,284 user reviews in the last 3...","- 88% of the 7,428,921 user reviews for this g...","Valve, Hidden Path Entertainment",Valve,"['English', 'Czech', 'Danish', 'Dutch', 'Finni...","['FPS', 'Shooter', 'Multiplayer', 'Competitive...","['Steam Achievements', 'Full controller suppor...",OS: | Windows® 7/Vista/XP | Processor: | Int...
2,Apex Legends™,"Apex Legends is the award-winning, free-to-pla...",Mixed,Very Positive,"- 65% of the 18,581 user reviews in the last 3...","- 80% of the 701,597 user reviews for this gam...",Respawn Entertainment,Electronic Arts,"['English', 'French', 'Italian', 'German', 'Sp...","['Free to Play', 'Multiplayer', 'Battle Royale...","['Online PvP', 'Online Co-op', 'Steam Achievem...",Requires a 64-bit processor and operating syst...
3,Forza Horizon 5,Your Ultimate Horizon Adventure awaits! Explor...,Very Positive,Very Positive,"- 87% of the 4,120 user reviews in the last 30...","- 88% of the 123,162 user reviews for this gam...",Playground Games,Xbox Game Studios,"['English', 'French', 'Italian', 'German', 'Sp...","['Racing', 'Open World', 'Driving', 'Multiplay...","['Single-player', 'Online PvP', 'Online Co-op'...",Requires a 64-bit processor and operating syst...
4,Call of Duty®,"Welcome to Call of Duty® HQ, the home of Call ...",Mixed,Mixed,"- 49% of the 8,257 user reviews in the last 30...","- 57% of the 236,876 user reviews for this gam...","Infinity Ward, Raven Software, Beenox, Treyarc...",Activision,"['English', 'French', 'Italian', 'German', 'Sp...","['FPS', 'Multiplayer', 'Shooter', 'Action', 'S...","['Single-player', 'Online PvP', 'Online Co-op'...",Requires a 64-bit processor and operating syst...
...,...,...,...,...,...,...,...,...,...,...,...,...
81187,The Front,The Front is a survival-open-world-crafting (S...,NaN,NaN,NaN,NaN,Samar Studio,Samar Studio,"['English', 'French', 'German', 'Russian', 'Ja...","['Early Access', 'Survival', 'Building', 'Sand...","['Single-player', 'MMO', 'Online PvP', 'LAN Pv...",OS: | Requires a 64-bit processor and operati...
81188,The Bornless,"Welcome to The Bornless, an action horror FPS....",NaN,NaN,NaN,NaN,Cathedral Studios,Cathedral Studios,"['English', 'French', 'Italian', 'German', 'Sp...","['Early Access', 'Horror', 'FPS', 'Shooter', '...","['Single-player', 'Online PvP', 'Online Co-op'...",OS: | Windows 10 or higher | Processor: | In...
81189,Winter Memories,After the unforgettable summer memories you’ve...,NaN,NaN,NaN,NaN,Dojin Otome,Kagura Games,"['English', 'Japanese', 'Simplified Chinese']","['Casual', 'RPG', 'Simulation', 'Sexual Conten...",['Single-player'],OS: | Windows® 7/8/8.1/10/11 | Processor: | ...
81190,Call of Duty®: Modern Warfare® III,In the direct sequel to the record-breaking Ca...,NaN,NaN,NaN,NaN,"Sledgehammer Games, Treyarch, Infinity Ward, B...",Activision,"['English', 'French', 'Italian', 'German', 'Sp...","['Action', 'FPS', 'Shooter', 'First-Person', '...","['Single-player', 'Online PvP', 'Online Co-op'...",Requires a 64-bit processor and operating syst...


In [5]:
full_df.isnull().sum()

Game                         19
Game Description            255
Recent Reviews Summary    21157
All Reviews Summary       75714
Recent Reviews Number     21157
All Reviews Number        75714
Developer                   250
Publisher                   578
Supported Languages           0
Popular Tags                  0
Game Features                 0
Minimum Requirements       1195
dtype: int64

In [6]:
rows_with_nan = full_df[full_df['All Reviews Summary'].isna()]

In [7]:
rows_with_nan.head()

,Game,Game Description,Recent Reviews Summary,All Reviews Summary,Recent Reviews Number,All Reviews Number,Developer,Publisher,Supported Languages,Popular Tags,Game Features,Minimum Requirements
6,Remnant II,Remnant II pits survivors of humanity against ...,Very Positive,NaN,"- 82% of the 19,384 user reviews for this game...",NaN,Gunfire Games,Gearbox Publishing,"['English', 'French', 'Italian', 'German', 'Sp...","['Souls-like', 'Action', 'Adventure', 'RPG', '...","['Single-player', 'Online Co-op', 'Steam Achie...",OS: | Win 10 | Processor: | Intel Core i5-76...
9,NARAKA: BLADEPOINT,Dive into the legends of the Far East in NARAK...,Mostly Positive,NaN,"- 75% of the 146,529 user reviews for this gam...",NaN,24 Entertainment,NetEase Games Global,"['English', 'Simplified Chinese', 'Traditional...","['Battle Royale', 'Multiplayer', 'Martial Arts...","['Online PvP', 'In-App Purchases', 'Partial Co...",OS: | Windows 7 64-bit or newer | Processor: ...
29,The Texas Chain Saw Massacre,Take on the role of one of the notorious Slaug...,Very Positive,NaN,"- 86% of the 1,834 user reviews for this game ...",NaN,Sumo Digital,Gun Interactive,['English'],"['Horror', 'Survival Horror', 'Gore', 'Team-Ba...","['Online PvP', 'Steam Achievements', 'Full con...",Requires a 64-bit processor and operating syst...
43,Starfield,Starfield is the first new universe in 25 year...,NaN,NaN,NaN,NaN,Bethesda Game Studios,Bethesda Softworks,"['English', 'French', 'Italian', 'German', 'Sp...","['Open World', 'Space', 'RPG', 'Singleplayer',...","['Single-player', 'Steam Achievements', 'Full ...",OS: | Windows 10 version 21H1 (10.0.19043) | ...
47,Sengoku Dynasty,Build and manage villages in an open-world feu...,Mostly Positive,NaN,"- 71% of the 1,345 user reviews for this game ...",NaN,Superkami,Toplitz Productions,"['English', 'French', 'German', 'Spanish - Spa...","['Early Access', 'Adventure', 'Life Sim', 'Sim...","['Single-player', 'Online Co-op', 'Full contro...",OS: | Windows 10 (x64) | Processor: | Intel ...
